# Adaptive LASSO Regression
One of the problems with [LASSO](lasso_regression.ipynb) is that the shrinkage in the variables is constant. `Adaptive LASSO` provides a solution by scaling the weights 

#### Background: Non-Negative Garrote (NNG)
The method that inspired `Adaptive LASSO`. We rewrite the `LASSO` problem as
$$min_{d_j} \sum_{i=1}^n \left(y_i - \sum_{j=1}^p d_j \hat{\beta}_j^{ols} x_{ij}\right)^2 + \lambda \sum_{j=1}^p d_j ~;~ d_j \ge 0$$

Where:
$$\hat{\beta}_j = \hat{d}_j \hat{\beta}_j^{ols}$$

So we are scaling the `OLS` solution.

The closed form solution in the `orthonormal` case looks like:
$$\beta_j^{garrote} = sign(\hat{\beta}_j^{ols}) \left(|\hat{\beta}_j^{ols}| - \frac{1}{2} \frac{\lambda}{|\hat{\beta}_j^{ols}|}\right)_+$$

---

`Adaptive LASSO` is very similar to `NNG`, but adds a small twist. The weights can be `OLS` estimates, `ridge` estimates, or any other estimates we want. It also has two tuning parameters:
* $\lambda$ — the usual regularization parameter
* $\gamma$  — tuning parameters for the weights

$$\beta^{Alasso} = argmin \|y - X\beta\|_2^2 + \lambda_n \sum_{j=1}^p \hat{w}_j|\beta_j|$$
Where $\hat{w}_j$ can be defined by:<br/>
$$\hat{w}_j = \frac{1}{|\hat{\beta}_j^{ols}|^{\gamma}}, \hat{w}_j = \frac{1}{|\hat{\beta}_j^{ridge}|^{\gamma}}$$

When $\gamma$ is small $\dots$

There's no direct `adaptive LASSO` implementation in `sklearn`, but one can alter the Lasso linear model.  

In [86]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import scipy as sp

# data parameters
n_samples, n_features = 500, 10000

# generate data
X, y, coef = make_regression(n_samples=n_samples, n_features=n_features, n_informative=50,
    noise=0.1, shuffle=True, coef=True, random_state=42)

scaler = StandardScaler()
X = scaler.fit_transform(X)

# lasso
# clf_lasso = Lasso(alpha=1., fit_intercept=False)
# clf_lasso.fit(X, y)
# coef_lasso = clf_lasso.coef_
# y_lasso = clf_lasso.predict(X)
# mse_lasso = np.sum((y - y_lasso) ** 2) / n_samples
# print(f"LASSO MSE: {mse_lasso: 0.2f}") 

# adaptive lasso
lambda_n, gamma = 0.01, 2
weights = np.ones(n_features)

obj = lambda beta: np.sum((y - X.dot(beta)) ** 2) \
                + lambda_n * 1 / (gamma) \
                * np.sum(np.power(np.abs(beta), gamma))

max_lasso_iterations = 5


for k in range(max_lasso_iterations):
    X_w = np.divide(X, weights[np.newaxis, :])
    
    clf_lasso = Lasso(alpha=1., fit_intercept=False)
    clf_lasso.fit(X_w, y)
    coef = clf_lasso.coef_ / weights
    weights = np.power(np.abs(coef), gamma)
    this_obj = obj(coef)
    print(weights)
    print(coef)
    print(this_obj)
    
# b_ols = np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))
# clf_ridge = Ridge(alpha=0., fit_intercept=False)
# clf_ridge.fit(X, y)
# b_ridge = clf_ridge.coef_
# w1 = 1. / np.abs(b_ols) ** gamma
# w2 = 1. / np.abs(b_ridge) ** gamma


# lasso = 
# # initialize 
# beta = np.zeros(n_features)

# obj = lambda b: (y - X.dot(b)) + lambda_n * np.sqrt(np.abs(b)) / np.sqrt(np.abs(b)) ** gamma

# n_lasso_iterations = 5
# # p_obj = lambda w: 1. / (2 * n_samples) * np.sum((y - np.dot(X, w)) ** 2) \
# #                   + alpha * np.sum(g(w))

# # obj = 
# y - X.dot(beta)
# obj(beta)

# for k in range(n_lasso_iterations):
    
#     clf = Ridge(alpha=lambda_n, fit_intercept=False)
#     clf.fit(X, y)
#     beta = clf.coef_ + lambda_n * beta * np.sqrt(np.abs(clf.coef_))
#     print(obj(beta))
    
#     beta = 
#     print(beta_hat)

[0. 0. 0. ... 0. 0. 0.]
[ 0.  0.  0. ... -0.  0.  0.]
702274.329202198


/Users/Ipanema/src/gstvolvr/notebooks/venv/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in true_divide
/Users/Ipanema/src/gstvolvr/notebooks/venv/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
nan
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
nan
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
nan
[nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
nan


In [78]:
# np.power(2, 2)

In [71]:
np.linalg.inv(X.T.dot(X))

array([[-4.73906715e+10, -1.44204607e+12,  1.01004427e+12, ...,
        -1.09315461e+12, -1.44065171e+12,  1.63715711e+12],
       [ 1.38555123e+12,  3.18984882e+12,  4.49321081e+12, ...,
         1.05019970e+12,  7.02268755e+12, -1.64279845e+12],
       [ 4.74946914e+11,  4.50926820e+11,  1.77214422e+12, ...,
        -1.55651407e+10,  1.96737158e+12,  1.11744986e+10],
       ...,
       [ 1.77300723e+11, -2.06580334e+12, -4.45934464e+11, ...,
        -1.76326910e+12, -3.37477630e+12,  2.80550137e+12],
       [ 9.83744435e+10, -1.45988123e+12,  1.10685249e+12, ...,
        -2.57129013e+12, -2.31474264e+12,  3.89655876e+12],
       [-1.41876401e+12, -3.98964033e+12, -2.20928711e+12, ...,
        -2.81432377e+12, -7.98859426e+12,  5.30809085e+12]])

In [72]:
np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))

array([ 1233792., -2224128.,   801984., ...,    87168.,   -42752.,
         267776.])

In [99]:
## credit: Alexandre Gramfort — https://gist.github.com/agramfort/1610922

import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import Lasso

n_samples, n_features = 500, 10000
X, y, coef = make_regression(n_samples=n_samples, n_features=n_features, n_informative=50,
    noise=0.1, shuffle=True, coef=True, random_state=42)

X /= np.sum(X ** 2, axis=0)

lambda_n = 0.01

g = lambda w: np.sqrt(np.abs(w))
gprime = lambda beta: 1. / (2. * np.sqrt(np.abs(beta)) + np.finfo(float).eps)

p = lambda beta: np.power(beta, gamma)
p_prime = lambda beta: gamma * np.power(beta, gamma - 1)

obj = lambda beta: np.sum((y - X.dot(beta)) ** 2) \
                + lambda_n * 1 / (gamma) \
                * np.sum(np.power(np.abs(beta), gamma))

weights = np.ones(n_features)
n_lasso_iterations = 10
train_score = np.zeros(n_lasso_iterations)

def loss(y, pred):
    return 1. / (2 * y.shape[0]) \
            * np.sum(np.square(y - pred)) \
            + lambda_n * np.sum(p(np.abs(coef)))
    
for k in range(n_lasso_iterations):
    X_w = X / weights.reshape(1, -1)
    clf = Lasso(alpha=lambda_n, fit_intercept=False)
    clf.fit(X_w, y)
    coef_ = clf.coef_ / weights
    weights = gprime(coef_)
    pred = X.dot(coef)
    train_score[k] = loss(y, pred)
    print(obj(coef_)) # should go down

204333631.98105326
240461623.17095697
240520635.75959608
240525705.67622042
240525610.83344257
240525610.79024196
240525610.79006127
240525610.79006013
240525610.7900602
240525610.7900602


In [100]:
train_score

array([98497.43727081, 98497.43727081, 98497.43727081, 98497.43727081,
       98497.43727081, 98497.43727081, 98497.43727081, 98497.43727081,
       98497.43727081, 98497.43727081])

In [121]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import Lasso

X, y, coef = make_regression(n_samples=306, n_features=8000, n_informative=50,
                    noise=0.1, shuffle=True, coef=True, random_state=42)

X /= np.sum(X ** 2, axis=0)  # scale features

alpha = 0.1
gamma = 2

# g = lambda w: np.sqrt(np.abs(w))
# gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)
# g = lambda w: np.power(w, gamma)
# # gprime = lambda w: gamma * np.power(w, gamma - 1 + np.finfo(float).eps)
# gprime = lambda w: gamma / (np.power(w, 1 - gamma) + np.finfo(float).eps)

# Or another option:
ll = 0.01
g = lambda w: np.log(ll + np.abs(w))
gprime = lambda w: 1. / (ll + np.abs(w))

n_samples, n_features = X.shape
p_obj = lambda w: 1. / (2 * n_samples) * np.sum((y - np.dot(X, w)) ** 2) \
                  + alpha * np.sum(g(w))

weights = np.ones(n_features)
n_lasso_iterations = 5

def loss(y, pred):
    return 1. / (2 * y.shape[0]) \
            * np.sum(np.square(y - pred)) \
            + lambda_n * np.sum(p(np.abs(coef)))

for k in range(n_lasso_iterations):
    X_w = X / weights[np.newaxis, :]
    clf = Lasso(alpha=alpha, fit_intercept=False)
    clf.fit(X_w, y)
    coef_ = clf.coef_ / weights
    preds = X.dot(coef_)
    weights = gprime(coef_)
    print(f'objective: {p_obj(coef_): 0.4f}')
    print(f'loss: {loss(y, preds): 0.4f}\n')

objective:  12255.7233
loss:  17115.6500

objective: -985.1588
loss:  3869.8042

objective: -985.2592
loss:  3869.6956

objective: -985.2608
loss:  3869.7064

objective: -985.2631
loss:  3869.7204



In [104]:
preds = X.dot(coef_)
loss(y, preds)

3872.356231517805

#### Resources:
* [ISyE8803: Topics on High-Dimensional Data Analytics — Module 4](https://courses.edx.org/courses/course-v1:GTx+ISYE8803+2T2019/course/#block-v1:GTx+ISYE8803+2T2019+type@chapter+block@223667a47f58432ea40f272c8ed71e11)
* [Alexandre Gramfort — Demo Adaptive LASSO](https://gist.github.com/agramfort/1610922)
* [Adaptive LASSO — sklearn pull request](https://github.com/henridwyer/scikit-learn/blob/c8b2c94e6335405e29fa411378d5d27c78808a6d/sklearn/linear_model/coordinate_descent.py)